# <center><u> Final LSTM/GRU Layout - Adri and Elian

In [84]:
#Import Libraries and Packages
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psycopg2
from dotenv import load_dotenv
load_dotenv()

True

In [85]:
#Access Credentials
db_host = os.getenv("DB_HOST")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_port = os.getenv("DB_PORT")

In [86]:
conn = psycopg2.connect(
    host=db_host,
    dbname=db_name,
    user=db_user,
    password=db_password,
    port=db_port
)

In [87]:
query="""
    select * 
    from agg.tidy_data_final
    where site = 20
"""

In [88]:
df = pd.read_sql_query(query, conn)

/var/folders/n_/s_vk866s6ds93jqhjv__r6cr0000gn/T/ipykernel_20951/1553155693.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [89]:
df = df.set_index('timestamp', drop=False)
df = df.sort_index()

# Prepare Data Set for LSTM/GRU Models

In [90]:
# Parameters

#n_input = no. of time steps considered before making the prediction
#n_features = no. of variables/features considered
#n_output = no. of time steps to forecast

# Note: 1 time step = 30 mins
# Look at the last two weeks to forecast the next 36 hours

# 1 day = 2 * 24 * 1 = 24
# 2 hours = 2 * 2 = 8 

n_input = 144
n_features = 14 #These includes the weather predictions that will be added later on and the normalised varaibles to be created
n_output = 72


# Selectfeatures of use for later on
df_input = df[['net_load', 'year', 'month', 'day', 'hour', 'day_of_week', 'season','precipitation_probability',
               'solar_radiation','sunshine_duration','weekend_or_bank_holiday','avg_net_load']]

# Normalise Data (as seen in Literature)

Group decision to use MaxAbs Scaler to normalise the data.

In [91]:
#Normalise all continuous variables needing scaling
from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()

variables = ['net_load','solar_radiation','sunshine_duration','avg_net_load']
for var in variables:
    new_var_name = var + '_norm'
    df_input[new_var_name] = scaler.fit_transform(df_input[[var]])

/var/folders/n_/s_vk866s6ds93jqhjv__r6cr0000gn/T/ipykernel_20951/4282254998.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input[new_var_name] = scaler.fit_transform(df_input[[var]])
/var/folders/n_/s_vk866s6ds93jqhjv__r6cr0000gn/T/ipykernel_20951/4282254998.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input[new_var_name] = scaler.fit_transform(df_input[[var]])
/var/folders/n_/s_vk866s6ds93jqhjv__r6cr0000gn/T/ipykernel_20951/4282254998.py:8: SettingWithCopyWarning: 
A value is trying to be

Now, we also want to include the predictions for 'solar radiation', 'sunshine_duration', 'precipitation_probability' as inputs.
These will be treated as new inputs (t+48) ~ 48 half-hourly timestamps representing day-ahead predictions.

In [92]:
df_input['sunshine_duration(t+48)'] = df_input['sunshine_duration_norm'].shift(-48) #Add prediction for time the day after
df_input['solar_radiation(t+48)'] = df_input['solar_radiation_norm'].shift(-48) #Add prediction for time the day after
df_input['precipitation_probability(t+48)'] = df_input['precipitation_probability'].shift(-48) #Add prediction for time the day after

/var/folders/n_/s_vk866s6ds93jqhjv__r6cr0000gn/T/ipykernel_20951/3763762080.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input['sunshine_duration(t+48)'] = df_input['sunshine_duration_norm'].shift(-48) #Add prediction for time the day after
/var/folders/n_/s_vk866s6ds93jqhjv__r6cr0000gn/T/ipykernel_20951/3763762080.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input['solar_radiation(t+48)'] = df_input['solar_radiation_norm'].shift(-48) #Add prediction for time the day after
/var/folders/n

In [93]:
#We want to drop last 48 rows as they will have Na values for the (t+48) columns/inputs.
df_input.dropna(inplace=True)

/var/folders/n_/s_vk866s6ds93jqhjv__r6cr0000gn/T/ipykernel_20951/1212278532.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input.dropna(inplace=True)


In [94]:
#Only Keep the Normalised Variables for the Model
df_input = df_input[['net_load_norm','year','month','day','hour','day_of_week','season','weekend_or_bank_holiday',
               'solar_radiation_norm','sunshine_duration_norm','avg_net_load_norm','sunshine_duration(t+48)',
               'solar_radiation(t+48)','precipitation_probability(t+48)']]

# Generate X and Y Matrices

In [95]:
X, y = [], []
for i in range(len(df_input) - n_input - n_output + 1):
    # Select all columns for the input sequence
    X.append(df_input.iloc[i:(i + n_input)].values)  
    
    y.append(df_input['net_load_norm'].iloc[(i + n_input):(i + n_input + n_output)].values)

X = np.array(X).reshape((len(X), n_input, -1))  
y = np.array(y)

# Train and Test Split

In [96]:
# Split into train (First ~330 days) and test (30 days)
# 1 time step = 30 mins

n_total_days = len(X) / (2 * 24)
n_test_days = 30
n_train_split = int((n_total_days - n_test_days) * 2 * 24)

X_train = X[:n_train_split]
y_train = y[:n_train_split]

X_test = X[n_train_split:]
y_test = y[n_train_split:]

# LSTM/GRU Sample Model

In [97]:
# Import all Necessary Keras Libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [98]:
#GRU Model - just change GRU for LSTM to retrain
#Model Architecture
model = Sequential()
model.add(GRU(256, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(GRU(128, activation='relu', return_sequences=True))
model.add(BatchNormalization())
model.add(GRU(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[1]))

In [99]:
#Compile Model
cp1 = ModelCheckpoint('final_model.keras', save_best_only=True)
model.compile(loss='mse', optimizer=Adam(learning_rate=0.001), metrics=[MeanSquaredError(), MeanAbsoluteError()])

In [ ]:
#Fit Model to Training Data with 10% Validation Split
history = model.fit(X_train, y_train, epochs=10, batch_size=16, validation_split=0.1)

# Random Search - Hyperparameter Tuning

Hyperparameters to tune:
* \# of LSTM/GRU units
* \# of LSTM/GRU layers
* Dropout Rate
* Batch Size

In [ ]:
def build_model(hp):
    model = Sequential()

    # Define first layer -> optimization for number of units and activation function
    model.add(Dense(units = hp.Int("units1",min_value = 32, max_value=1024, step=16),
                   activation = hp.Choice("activation1",["relu","tanh"]),
                    input_shape = (X_train_scaled.shape[1],)
                   ))

    # Define dropout layer by providing the optimization of dropout rate
    model.add(Dropout(hp.Float("rate1",min_value=0.1, max_value = 0.4, step=0.1),seed = 12345))

    # Add second layer
    model.add(Dense(units = hp.Int("units2",min_value = 32, max_value=128, step=16),
                   activation = hp.Choice("activation2",["relu","tanh"])
                   ))


    model.add(Dropout(hp.Float("rate2",min_value=0.1, max_value = 0.4, step=0.1),seed = 12345))

    model.add(Dense(units = hp.Int("units3",min_value = 32, max_value=128, step=16),
                   activation = hp.Choice("activation3",["relu","tanh"])
                   ))
    # Output Layer
    model.add(Dense(units=1, activation='sigmoid'))

    # define values for learning rate
    learning_rate = hp.Float("learning_rate",min_value = 0.001, max_value=0.1, step=0.01)

    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss = 'binary_crossentropy',
                  metrics = ['accuracy']
                 )
    return model

In [ ]:
import keras_tuner as kt
build_model(kt.HyperParameters())

In [ ]:
rtuner = kt.RandomSearch(hypermodel=build_model,
    objective="val_accuracy",
    max_trials=5,
    seed=12345,
    overwrite=True,
    directory="my_dir",
    project_name="diab")

In [ ]:
rtuner.search(X_train,
              y_train,
              epochs=10,
              validation_data=(X_test, y_test))

In [ ]:
tuned_model = rtuner.get_best_models(num_models=1)

In [ ]:
tuned_model.summary()

In [ ]:
rtuner.get_best_hyperparameters().get_config()['values']